# Deep Q-learning
##### Authors: Eirik Fagtun Kjærnli and Fabian Dietrichson [Accenture]

Deep Reinforcement learning has resulted in superman performance in a range of scenarios, such as Chess, Atari 2600, and Starcraft, but also in robotics. The main reason for this success compared to traditional Reinforcement Learning is the introduction of deep neural networks. The first successful implementation of deep neural networks in combination with a Reinforcement learning algorithm was Mnih et. al. which was able to reach superhuman performance in the classic Atari games. Their approach was coined Deep Q-Networks(DQN), which is very similar to the Q-learning algorithm we implemented in the previous workshop, in combination with deep neural networks. This is the algorithm we will be implementing in this notebook.

In the DQN approach, a neural network replaces the Q-table which we used in the previous notebook. The main reason for this is the neural networks ability to generalize around similar states. In a continuous environment, a Q-table would quickly grow to tens of millions of unique states, although many of these are so similar that the same action should be applied. This is both an issue due to the huge table we would need, but also because we would need a tremendous amount of training data to update all the states. 

Neural networks address this issue, however, they do come with their own challenges. Neural network struggles with instabilities during training, in addition to critical forgetting and divergence. Mnih's success was due to several ingenious tricks which stabilized the network during training, and we will go through these during the workshop.

In the previous notebook, we created a lot of methods, which resulted in a lot of parameters being passed from method to method. This can quickly become convoluted, and in this notebook we instead use classes. An example can be seen below. When the class is created the init method is run, and the different parameters are set and stored in this instance of the class. The parameters can be called within the class using "self.<name_of_parameter>" syntax, e.g. self.learning_rate.

## Import packages and create support methods for workshop
Before you can go on, the cell below must be run. These are methods used to verify your work, in addition to the support function that will be used throughout the workshop. 

### Task 1
Import the packages needed for this workshop, simply mark the cell below and press CTRL + Enter

### Task 2
Create the necessary support methods by running the second cell below. Mark it and press CTRL + Enter

In [ ]:
import warnings
import logging
import gym
import os
import datetime
from statistics import mean
import numpy as np
import random
from copy import deepcopy
from collections import deque, namedtuple

from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.metrics import MSE
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.initializers import glorot_uniform

warnings.filterwarnings("ignore")
logging.getLogger('tensorflow').disabled = True
%precision 3
should_assert = True

In [ ]:
def generate_blank_weights(layer_dims):
    
    # Xavier/Glorot Initialization
    new_weights = np.random.randn(layer_dims[0], layer_dims[1])*np.sqrt(1/layer_dims[0])
    new_bias = np.zeros((layer_dims[1]))
    
    return [new_weights, new_bias]

def create_dummybuffer():
    parameters = {"buffer_size": 6,
                  "batch_size": 3}
    DummyBuffer = ExperienceReplay(parameters)
    
    return DummyBuffer

def get_dummy_parameters_and_env():
        
        parameters = {
            "tau" : 0.4,
            "gamma" : 1,
            "epsilon_init" : 1,
            "epsilon_decay" : 1,
            "epsilon_minimum": 0.01,
            "buffer_size" : 2000,
            "batch_size" : 64,
            "epochs": 1,
            "loss_metric" : "mse",
            "learning_rate" : 0.01,
            "learning_rate_decay": 0.01,
            "hidden_layer_1": 5,
            "hidden_layer_2": 5}
        
        dummy_env = gym.make("CartPole-v0")
        return parameters, dummy_env

## Create Q-Networks
There are several frameworks which can be used to create neural networks such as Tensorflow from Google, PyTorch from Facebook or MXNet from Apache. In this notebook, we will be using Tensorflow, with a high-level API called Keras on top. Keras greatly simplifies the effort needed to create and train a neural network, and it is often referred to as the best Deep Learning framework for those who are just starting with neural networks.

Neural networks are partially inspired by the structure of the human brain and consist of a network of interconnected neurons. The neural network consists of an input layer, a set of hidden layers, and an output layer, as shown in the figure below.

<img src="Images/Neural_network__achitecture.svg" alt="drawing" width="400" height="200"/>



### Task
Now we will see how easy it is to create a neural network using Keras. We are going to create a neural network with an input layer, two hidden layers, and one output layer. The design criterions are as follows: <br>

1. The model should be fully connected 
<br>
<br>
2. The input layer and first hidden layer is created together, and should:
    - Be of type Dense
    - Use the ReLU activation function
    - Have input size equal to the observations size of the environment
    - Have hidden layers size equal to parameter hidden_layer_1 
<br>
<br>
3. The second hidden layer should:
    - Be of type Dense
    - Use the ReLU activation function 
    - The hidden layer should have size equal to parameter hidden_layer_2
    - Tip1: This layer is similar to previous, but without the input_dim parameter
    - Tip2: Remember to add the dense layer using model.add()
<br>
<br>
4. The output layer should:
    - Be of type Dense
    - The activation function should be linear
    - Should have size equal to action vector, i.e. action_size
<br>
<br>
5. The final part is to compile the network. Before we do this we must define som parameters
    - Loss metric should be Mean-squared-error (MSE)
    - The optimizer should be Adaptive Moment Estimation (Adam)
    - Learning rate should equal to learning_rate
    - Learning rate decay should be equal to learning_rate_decay
<br>
As you can see below step 1,2 and 5 is already done, so your task is to complete task 3 and 4.

**Ps!** <br>
At the end of the workshop, you can experiment with different structures and parameters, however, in the first walkthrough, you will follow the defined steps.

In [ ]:
class QNetwork:

    def __init__(self, env, parameters):
        self.observations_size = env.observation_space.shape[0]
        self.action_size = env.action_space.n
        self.learning_rate = parameters["learning_rate"]
        self.learning_rate_decay = parameters["learning_rate_decay"]
        self.loss_metric = parameters["loss_metric"]
        self.hidden_layer_1 = parameters["hidden_layer_1"]
        self.hidden_layer_2 = parameters["hidden_layer_2"]    
        
    def build_q_network(self):
        
        model = Sequential()
        model.add(Dense(self.hidden_layer_1, input_dim=self.observations_size, activation='relu'))
        
        "Input code below"
        
        "Input code above"
        
        model.compile(loss=self.loss_metric, optimizer=Adam(lr=self.learning_rate, decay=self.learning_rate_decay))
    
        return model
    

In [ ]:
# Do not edit - Assertion cell #
if(should_assert):
    mock_parameters = {"loss_metric" : "mse",
                "learning_rate" : 0.01,
                "learning_rate_decay": 0.01,
                "hidden_layer_1": 24,
                "hidden_layer_2": 24}

    mock_env = gym.make("CartPole-v0")
    mock_q_network = QNetwork(mock_env, mock_parameters)
    mock_model = mock_q_network.build_q_network()
    config_network = mock_model.get_config()
    assert(config_network.get("name")[0:10] == "sequential")

    # Layers
    config_layers = config_network.get("layers")
    assert(len(config_layers) == 3), "You should only have 2 dense layers"

    # First layer
    assert(config_layers[0].get("class_name") == "Dense"),"Incorrect layertype in first layer"
    assert(config_layers[0].get("config").get("batch_input_shape") == (None, mock_q_network.observations_size))
    assert(config_layers[0].get("config").get("units") == 24), "Incorrect number of neurons in first layer"
    assert(config_layers[0].get("config").get("activation") == "relu"), \
    "Activation function for first layer should be relu"

    # Second layer
    assert(config_layers[1].get("class_name") == "Dense"),"Incorrect layertype in first layer"
    assert(config_layers[1].get("config").get("units") == 24), "Incorrect number of neurons in first layer"
    assert(config_layers[1].get("config").get("activation") == "relu"), \
    "Activation function for second layer should be relu"

    # Thrid layer
    assert(config_layers[2].get("class_name") == "Dense"),"Incorrect layertype in first layer"
    assert(config_layers[2].get("config").get("units") == mock_q_network.action_size), "Incorrect number of neurons in first layer"
    assert(config_layers[2].get("config").get("activation") == "linear"),\
    "Activation function for third layer should be linear"

    config_optimizer = mock_model.optimizer.get_config()
    assert(0.0099 < config_optimizer.get("lr") <= 0.01),"Learning rate should be 0.01"
    assert(0.0099 < config_optimizer.get("decay") <= 0.01),"Learning rate decay should be 0.01"
    assert(mock_model.loss == "mse"), "Loss metric should me mse"
    
    print("Superb, you implemented the layers correct! Information about your model is shown below.\n")
    print("PS. The input layer is not shown, altough you should be able to calculate it by looking at <Param #>")
    print(" of the first hidden layer.\n")
    mock_model.summary()
    

## Create an Experience Buffer

For a neural network to perform optimally we want the data to be I.I.D (Independent and Identically Distributed). In supervised learning, where for instance the network is fed an image and it will predict either cat or dog, this is achieved by randomly sampling the training data from the full data set. As a result:
- Batches have close-to similar data distribution.
- Samples in each batch are independent of each other.

In Reinforcement learning where our agent samples data by moving from state to state, the recently sampled data will be highly correlated to each other. As a result, we will feed our network data which closely resembles each other, which is a recipe for disaster when working with a neural network. 

Furthermore, the data distribution of the initial states will be different from that of the later stage, i.e. this does not satisfy the I.I.D criterion. This is bad news!

Luckily Mnih et. al. has a solution. Instead of training on the just the previously collected samples, we store all states in an **Experience Buffer** and sample randomly from this buffer.

### Warm-up
The Experience Buffer is a collection of all the experiences the agent has collected during its interaction with the environment. As a result, the experience buffer is empty at the begin of the first episode. This is not an issue, as you in the previous task implemented the necessary logic to handle the case when there are fewer experience_touples in the buffer than the batch size. However, this does slow down training as we train with few and highly correlated samples. 

To counteract this issue, we introduce a method called warm_up. The warm_up method performs completes a certain amount of episodes taking random actions, thereby creating reducing the correlation between samples in the initial training phase.

## Tasks

This section is divided into multiple tasks. You will first complete task 1, and then run the first assertion cell for task 1. Then you will continue to task 2 and run assertion cell 2 etc.

### Task 1

The first method we are going to create is a method which adds an experience, i.e. a state-transition. This is stored as a tuple which contains the following fields; state, action, reward, done, new_state.

To store a state-transition we are going to use a specific type of tuple, namely a namedtuple. You can read more about it [here](https://docs.python.org/2/library/collections.html#collections.namedtuple). We have created a namedtuple which you are going to use called experience_tuple. The experience_tuple have the following field where you can store information; state, action, reward, done, new_state. To create a state-transition do the following:
- self.experience_tuple(state=your_state, action=your_action, reward=your_reward, done=your_done, new_state=your_new_state)

To add a named_tuple to the buffer use the append method of the experience_buffer:
- self.experience_buffer.append(your_experience_tuple)


### Task 2

The next method we are going to create is a method which returns a batch of experience_tuples, which is used during training. The methods should set the variable *experiences* equal to either
- All the samples in the experience_buffer IF it is less than the batch size
- or ELSE a *batch_size* of randomly sample experience_tuples from the experience_buffer. To randomly sample use the random.sample() method. You can read more about the method [here](https://docs.python.org/2/library/random.html).

The last part of the method is created by us, and it creates vectors of all elements of each field. This is done to vectorize the training and gives a significant reduction in training time.

### Task 3

The final method we will create is the warm-up method we introduced earlier. We have created the high-level structure, but you will create the code which completes one episode. The method should do the following
1. Use a While-loop which runs as long as the Done parameter is "not true"
2. Chooses a random action - Use env.action_space.sample()
3. Use the random action to make a step in the environment - Use env.step(random_action).
4. Add the experience to the replay_buffer - Use the add_experience method we created earlier
5. Remember to update the current state, to the new_state, e.g. state = new_state

Remember to test your method by running the assertion cell for task 3.

In [ ]:
class ExperienceReplay:
    
    def __init__(self, parameters):
        self.buffer_size = parameters["buffer_size"]
        self.batch_size = parameters["batch_size"]
        self.experience_buffer = deque(maxlen=self.buffer_size)
        
        self.experience_tuple = namedtuple("Experience", 
                                           field_names=["state", "action", "reward", "done", "new_state"])
    
    def add_experience(self, state, action, reward, done, new_state):
        
        "Input code below"
        
        "Input code above"
        
    def get_batch(self):
        
        "Input code below"
        
        "Input code above"
        
        
        states = np.vstack([e.state for e in experiences if e is not None])
        actions = np.vstack([e.action for e in experiences if e is not None])
        rewards = np.vstack([e.reward for e in experiences if e is not None])
        new_states = np.vstack([e.new_state for e in experiences if e is not None])
        dones = np.vstack([e.done for e in experiences if e is not None])
        
        return (states, actions, rewards, dones, new_states)
    
    def warm_up(self, env):
        for _ in range(10):
            state = env.reset()
            done = False
            
            "Input code below"
            
            "Input code above"

### Assertion - Task 1

In [ ]:
# Do not edit - Assertion cell #
if should_assert:
    DummyBuffer = create_dummybuffer()

    for i in range(6):
        DummyBuffer.add_experience(i, i%2, i, i%2, i+1)

    dummy_experience_buffer = DummyBuffer.experience_buffer.copy()
    assert(len(dummy_experience_buffer) == 6),\
    "Length of your experience buffer is wrong, should be 6, was {}".format(len(dummy_experience_buffer))

    for e in range(6):
        dummy_experience = DummyBuffer.experience_tuple(state=e, action=e%2, reward=e, done=e%2, new_state=e+1)
        stored_experience = dummy_experience_buffer.popleft()
        assert(dummy_experience == stored_experience), \
        "The values were incorrectly stored, was {}, should have been {}".format(stored_experience, dummy_experience)

    print("You implemented the add_experience method correctly, great job!")

### Assertion - Task 2

In [ ]:
# Do not edit - Assertion cell #
if should_assert:
    DummyBuffer = create_dummybuffer()
    for i in range(2):
        DummyBuffer.add_experience(i, i%2, i, i%2, i+1)

    assert(len(DummyBuffer.get_batch()[0]) == 2),\
    "Should return 2 samples, when there are less experience_tuples than the batch_size. Your method returned {} samples"\
    .format(len(DummyBuffer.get_batch()[0]))

    DummyBuffer.add_experience(3, 3%2, 3, 3%2, 3+1)
    dummy_batch = DummyBuffer.get_batch()
    assert(len(dummy_batch[0]) == 3),\
    "Should return 3 samples, when there are 3 experience_tuples in the experience_buffer. Your method returned {} samples"\
    .format(len(DummyBuffer.get_batch()[0]))


    DummyBuffer.add_experience(4, 4%2, 4, 4%2, 4+1)
    dummy_batch = DummyBuffer.get_batch()
    assert(len(dummy_batch[0] == 3)),\
    "Should return 3 samples, when there are more experience_tuples than the batch_size. Your method returned {} samples"\
    .format(len(DummyBuffer.get_batch()[0]))

    assert(len(np.unique(dummy_batch[0][0:3])) == 3), "The method did not return the values correctly, states should be all unique values. You returned {}".format(dummy_batch[0])
    assert(len(np.unique(dummy_batch[-1][0:3])) == 3), "The method did not return the values correctly, new_states should be all unique values. You returned {}".format(dummy_batch[-1])

    print("Great, you implemented the get_batch() method correctly")

### Assertion Task 3

In [ ]:
# Do not edit - Assertion cell #
if should_assert:
    parameters = {"buffer_size": 1000,
                  "batch_size": 1}
    mock_env1 = gym.make("CartPole-v1")
    mock_experience_replay = ExperienceReplay(parameters)
    mock_env1.reset()
    mock_env2 = deepcopy(mock_env1)

    mock_experience_replay.warm_up(mock_env1)
    mock_state = mock_env2.reset()
    while 0 < len(mock_experience_replay.experience_buffer):
        mock_action = mock_env2.action_space.sample()
        mock_new_state, mock_reward, mock_done, _ = mock_env2.step(mock_action)
        
        mock_experience_tuple = mock_experience_replay.experience_buffer.popleft()
        np.testing.assert_array_equal(mock_experience_tuple[0], mock_state, "State was incorrect")
        assert(mock_experience_tuple[1] == mock_action), "Action was incorrect"
        assert(mock_experience_tuple[2] == mock_reward), "Reward was incorrect"
        np.testing.assert_array_equal(mock_experience_tuple[4], mock_new_state, "New_state was incorrect")
        
        if mock_done:
            mock_state = mock_env2.reset()
        else:
            mock_state = mock_new_state
            
    print("Your warm-up method seems to function properly, good job!")

## Create Agent

### Target network

In the supervised learning example, where we are to predict cat or dog, the labels are stationary, e.g. an image of a cat will always have the label cat. This is not the case in Reinforcement learning.

As we experienced in the previous notebook, the Q-value of the next states is calculated based on the assumption that the Q-table is correct. However, since we begin each training session with a blank table and update the values as we go along, the Q-values of the next state is just an estimate. As a result, we are chasing a moving target, which degrades the training of the neural network and may lead to divergence.

Mnih et. al. solved this by using a target network to choose action and to predict the Q-values of the next state $S_{t+1}$, and used a local network to predict the Q-values of state $S_{t}$. The two networks have an identical structure. To update the target network, the equation below is used, where $\theta$ represents the weights of each network.

$
\theta_{target} = \tau * \theta_{local} + (1-\tau) * \theta_{target}
$

### Task 1
Implement the target network algorithm in the "update_target_network" method below. We have already created the shell, where we loop through each layer of the network. Your task
- Update the weights of each layer,  by using the algorithm above. Remember to use the self.tau parameter.
- Remember to run the assertion cell

### Q-learning algorithm


### Task 2
Go through the Q-learning algorithm, and compare it to the Q-learning algorithm we saw in the last notebook. Make sure you understand the implementation before moving on.


In [ ]:
class Agent:    

    def __init__(self, env, parameters):
        self.env = env
        self.q_network = QNetwork(env, parameters)
        self.local_network = self.q_network.build_q_network()
        self.target_network = self.q_network.build_q_network()
        self.experience_replay = ExperienceReplay(parameters)
        
        self.epsilon = parameters["epsilon_init"]
        self.epsilon_decay = parameters["epsilon_decay"]
        self.epsilon_minimum = parameters["epsilon_minimum"]
        self.tau = parameters["tau"]
        self.gamma = parameters["gamma"]
        self.epochs = parameters["epochs"]

    def update_local_network(self):
        states, actions, rewards, dones, next_states = self.experience_replay.get_batch()
        
        # Get Q-values for the next state, Q(next_state), using the target network
        Q_target = self.target_network.predict(next_states)

        # Apply Q-learning algorithm and Q-value for next state to calculate the actual Q-value the Q(state)
        Q_calc = rewards + (self.gamma * np.amax(Q_target, axis=1).reshape(-1, 1) * (1 - dones))
        
        # Calculate Q-value Q(state) we predicted earlier using the local network
        Q_local = self.local_network.predict(states)
        
        # Update Q_values with "correct" Q-values calculated using the Q-learning algorithm      
        for row, col_id in enumerate(actions):
            Q_local[row, np.asscalar(col_id)] = Q_calc[row]
        
        # Train network by minimizing the difference between Q_local and modified Q_local
        self.local_network.fit(states, Q_local, epochs=self.epochs, verbose=0)
    
    def update_target_network(self):
        local_weights = self.local_network.get_weights()
        target_weights = self.target_network.get_weights()

        for layer in range(len(local_weights)):
            
            "Input code below"
        
            "Input code above"
        
        self.target_network.set_weights(target_weights)
    
    def update_epsilon(self):
        if self.epsilon >= self.epsilon_minimum:
            self.epsilon *= self.epsilon_decay
        
    def select_action(self, state):
    
        if self.epsilon > np.random.uniform():
            action = env.action_space.sample()
        else:
            action = np.argmax(self.local_network.predict(np.array([state])))

        return action
    
    def step(self, env, state):
        
        action  = self.select_action(state)
        new_state, reward, done, _ = env.step(action)

        return action, reward, done, new_state
    
    def save(self):
        save_dir = os.path.join(os.getcwd(), env.spec.id +"_"+ datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
        os.makedirs(save_dir)
        self.target_network.save("target_network.h5")
        self.local_network.save("local_network.h5")
        print("Weights saved successfully")
        
    def load(self):
        uri = "freeze_weights/" + env.spec.id + "/"
        self.target_network.load_weights(uri + "target_network.h5")
        self.local_network.load_weights(uri + "local_network.h5")
        print("Weights loaded successfully for environment: {}".format(env.spec.id))
        
    def freeze_network(self, freeze_layers):
        # Freezes first freeze_layers of the network and resets the other if freeze_layers is not zero.
        network_size = len(self.local_network.get_config().get("layers"))
        assert(freeze_layers <= network_size),\
        "Tried to freeze more layers than there are layers in local network!"
        
        self.load()
        
        if freeze_layers == 0:
            print("No layers frozen, using loaded weights")
        elif freeze_layers == network_size:
            for layer in range(network_size):
                self.local_network.layers[layer].trainable = False 
            print("Frozen all layers, and using loaded weights")
        else:             
            for layer in range(network_size):
                if layer < freeze_layers:
                    self.local_network.layers[layer].trainable = False 
                else:
                    new_weights = generate_blank_weights(self.local_network.layers[layer].get_weights()[0].shape)
                    self.local_network.layers[layer].set_weights(new_weights)
                    self.target_network.layers[layer].set_weights(new_weights)    
            
            self.local_network.compile(loss=self.q_network.loss_metric, 
                          optimizer=Adam(lr=self.q_network.learning_rate, 
                                         decay=self.q_network.learning_rate_decay))
            print("Networks first {} layers are succesfully frozen".format(freeze_layers))
            

In [ ]:
# Do not edit - Assertion cell #
if should_assert:
    dummy_parameters, dummy_env = get_dummy_parameters_and_env()
    dummy_dqn = Agent(dummy_env, dummy_parameters)

    dummy_local_weights = deepcopy(dummy_dqn.local_network.get_weights())
    dummy_target_weights = deepcopy(dummy_dqn.target_network.get_weights())

    dummy_dqn.update_target_network()
    for i in range(len(dummy_target_weights)):
        np.testing.assert_array_equal(
            dummy_dqn.target_network.get_weights()[i], 
            dummy_parameters.get("tau") * dummy_local_weights[i] + (1 - dummy_parameters.get("tau")) * dummy_target_weights[i],
            err_msg="\nThe target network was not implemented correctly. Layer {} was incorrect\n".format(i))

    print("Great job, you implemented the update_target_network-method correctly!")


## Train

The training method is more or less identical to the method we used in the previous notebook, with some modifications due to the introduction of a neural network to represent the Q-table.


### Task
Make sure you understand the method before moving on!
- Do you understand why the episodic reward is a better metric of the neural network's performance than the loss-metric when doing Reinforcement learning?


In [ ]:
def train(agent, env, iterations, episodes):
    
    total_reward = 0
    total_reward_list, iterations_list = [], []
    agent.experience_replay.warm_up(env)
    
    for episode in range(episodes):
        
        state = env.reset()
        total_reward=0
        
        if (episode != 0): 
            agent.update_epsilon()
    
        for iteration in range(iterations):
            
            action, reward, done, new_state = agent.step(env, state)
            agent.experience_replay.add_experience(state, action, reward, done, new_state)
            
            state = new_state
            
            agent.update_local_network()
            agent.update_target_network()
            total_reward += reward
            
            if done: 
                break
        
        total_reward_list.append(total_reward)
        iterations_list.append(iteration+1)
        
        if episode % 10 == 0 and episode != 0:
            print("Episode: {0:d}-{1:d} | Average iterations: {2:0.2f}  | Average total reward: {3:0.2f} | Epsilon: {4:0.4f}" \
                  .format(episode-10, episode, mean(iterations_list), mean(total_reward_list), agent.epsilon))
            total_reward_list.clear()
            iterations_list.clear()
        
    return agent

## Play
The logic in the play method is identical to the method we used in the previous method.
The method completes one episode using the inputted agent-object.

In [ ]:
def play(agent):
    
    done = False
    agent.epsilon = 0
    total_reward = 0
    env = agent.env
    state = env.reset()
    
    while not done:
        action, reward, done, new_state = agent.step(env, state)
        state = new_state
        
        total_reward += reward
    
    print("Total Reward: {}".format(total_reward))

## Hyperparameters

The hyperparameters are the customizable parameters which are not learned by the agent but are set by us.

The number of parameters passed to the agent significantly increases when we begin using neural networks. To simplify the process, we define them in a dictionary and simply pass the dictionary to the agent. We have chosen the hyperparameters for some environments, and they should not be changed the first time you go through the notebook.

PS. You may change them as you please when you have passed through the notebook once.

In [ ]:
def get_hyperparameters(env):
    
    env_id = env.spec.id

    if env_id == "CartPole-v0":
        print("Hyperparameters for {} chosen!".format("CartPole-v0"))
        parameters = {
            "tau" : 0.05,
            "gamma" : 0.95,
            "epsilon_init" : 1,
            "epsilon_decay" : 0.97,
            "epsilon_minimum": 0.01,
            "buffer_size" : 10000,
            "batch_size" : 32,
            "epochs": 1,
            "loss_metric" : "mse",
            "learning_rate" : 0.01,
            "learning_rate_decay": 0.01,
            "hidden_layer_1": 24,
            "hidden_layer_2": 24}
        
    elif env_id == "CartPole-v1":
        print("Hyperparameters for {} chosen!".format("CartPole-v1"))
        parameters = {
            "tau" : 0.05,
            "gamma" : 0.99,
            "epsilon_init" : 1,
            "epsilon_decay" : 0.99, #0.999
            "epsilon_minimum": 0.01,
            "buffer_size" : 2000,
            "batch_size" : 64,
            "epochs": 1,
            "loss_metric" : "mse",
            "learning_rate" : 0.01,
            "learning_rate_decay": 0.01,
            "hidden_layer_1": 24,
            "hidden_layer_2": 24}
        
    elif env_id == "LunarLander-v2":
        print("Hyperparameters for {} chosen!".format("LunarLander-v2"))
        parameters = {
            "tau" : 0.05,
            "gamma" : 0.99,
            "epsilon_init" : 1,
            "epsilon_decay" : 0.999,
            "epsilon_minimum": 0.01,
            "buffer_size" : 2500,
            "batch_size" : 64,
            "epochs": 1,
            "loss_metric" : "mse",
            "learning_rate" : 0.01,
            "learning_rate_decay": 0.01,
            "hidden_layer_1": 24,
            "hidden_layer_2": 24}
    else:
        print("Standard hyperparameters chosen!")
        parameters = {
            "tau" : 0.05,
            "gamma" : 0.95,
            "epsilon_init" : 1,
            "epsilon_decay" : 0.97,
            "epsilon_minimum": 0.01,
            "buffer_size" : 10000,
            "batch_size" : 32,
            "epochs": 1,
            "loss_metric" : "mse",
            "learning_rate" : 0.01,
            "learning_rate_decay": 0.01,
            "hidden_layer_1": 24,
            "hidden_layer_2": 24}
    
    return parameters

## Explore Environment


In [ ]:
environment = "CartPole-v1"
episodes = 5
iterations = 500
layers_to_freeze = 1

env = gym.make(environment)
parameters = get_hyperparameters(env)

dqn_agent = Agent(env, parameters)
dqn_agent.freeze_network(layers_to_freeze)
dqn_agent = train(dqn_agent, env, iterations, episodes)

## Test trained agent for CartPole-v1
For the CartPole-v0 environment the maximum score is 500.

In [ ]:
play(dqn_agent)

## Run - Record - Show : One simulation

In [ ]:
env = gym.make("CartPole-v1")
monitor = gym.wrappers.Monitor(env, directory="videos", force=True)

for _ in range(3):
    play(dqn_agent, monitor)

monitor.close()
env.close()

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
#
""".format("./videos/"+list(filter(lambda s: s.endswith(".mp4"), os.listdir("./videos/")))[-1]))